In [1]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = 'notebook'  # или 'browser', 'colab', 'jupyterlab '

In [2]:
df=pd.read_parquet('data/data.parquet')

In [6]:
# Заголовок дашборда
header = widgets.HTML("<h1>📊 Отмены</h1>")
display(header)

# Создаем виджеты
dropdown1 = widgets.Dropdown(
    options=['Все'] + sorted(list(df['Месяц'].unique())),
    description='Месяц:',
    value='Все'
)

dropdown2 = widgets.Dropdown(
    options=['Все'] + sorted(list(df['Направление'].unique())),
    description='Направление:',
    value='Все'
)

dropdown3 = widgets.Dropdown(
    options=[],
    description='Станция:',
    disabled=True
)

# Создаем выход для графика и таблицы
graph_output = widgets.Output()
table_output = widgets.Output()

def update_stations(change):
    """Обновляет список станций при изменении направления"""
    if dropdown2.value != 'Все':
        available_stations = df[df['Направление'] == dropdown2.value]['Станция'].unique()
        dropdown3.options = ['Все'] + sorted(list(available_stations))
        dropdown3.disabled = False
    else:
        dropdown3.options = []
        dropdown3.disabled = True
    update_visualization(None)

def update_visualization(change):
    with graph_output:
        graph_output.clear_output(wait=True)
        
        # Получаем текущие значения
        month = dropdown1.value
        direction = dropdown2.value
        station = dropdown3.value if not dropdown3.disabled else None
        
        # Фильтруем данные
        filtered_df = df.copy()
        
        if direction != 'Все':
            filtered_df = filtered_df[filtered_df['Направление'] == direction]
            
            if station and station != 'Все':
                filtered_df = filtered_df[filtered_df['Станция'] == station]
        
        # Группируем по месяцам и суммируем количество
        grouped_data = filtered_df.groupby('Месяц', as_index=False)['Количество'].sum()
        
        # Создаем интерактивный график
        fig = px.bar(
        grouped_data,
        x='Месяц',
        y='Количество',
        text='Количество',
        title=f"Отмены {'по станции ' + str(station) if station and station != 'Все' else 'по всем станциям'}" + 
             f"{' за ' + str(month) if month != 'Все' else ' по месяцам'}" +
             f"{' (Направление: ' + str(direction) + ')' if direction != 'Все' else ''}",
        color_discrete_sequence=["#6a35be"]
        # color='Количество',  
        # color_continuous_scale='cividis' 
        )

        # Настраиваем внешний вид
        fig.update_traces(texttemplate='%{text:,}', textposition='outside')
        fig.update_layout(
            width=800,
            height=500,
            uniformtext_minsize=8,
            uniformtext_mode='hide',
            yaxis_title='Количество отмен',
            xaxis_title='Месяц',
            hovermode='x unified'
        )
        
        # Если выбран конкретный месяц, показываем только один столбец
        if month != 'Все':
            fig.update_xaxes(range=[month, month])
        
        display(fig)
    
    with table_output:
        table_output.clear_output(wait=True)
        if not filtered_df.empty:
            display(filtered_df.groupby('Месяц', as_index=False)['Количество'].sum().style.format({'Количество': '{:,}'}))

# Подписываемся на изменения
dropdown1.observe(update_visualization, names='value')
dropdown2.observe(update_stations, names='value')
dropdown3.observe(update_visualization, names='value')

# Отображаем все элементы управления
controls = widgets.VBox([
    dropdown1,
    dropdown2,
    dropdown3
])

display(controls)
display(graph_output)
display(table_output)

# Инициализируем вывод
update_visualization(None)

HTML(value='<h1>📊 Отмены</h1>')

Output()

Output()